In [ ]:
import sys
sys.path.append('../')
from cyto_ml.data.vectorstore import vector_store

By now we should have a vector db (chromadb for now, sqlite3 under the hood) full of 2048-long lists of image embeddings.
What can we hope to drop out of them?

* Image similarity search (either on an image in this collection, or an unseen one)
* Different self-supervised clustering methods (nice reference here: https://sslneurips23.github.io/paper_pdfs/paper_70.pdf)

What are the outcomes we are looking for here (if any of this in fact works?)

* Insights into assemblies of functional traits without having to do much taxonomy, very open-ended
* Ability to train a cheap-to-run binary classifier (plankton or not) that reliably filters data before it goes into object storage, without having to develop many rules
* Ability to gauge how well an off the shelf model is able to discriminate our data (assess the value of doing a lot of labelling for a custom model)
* Utility for assisted labelling without having to do _much_ ML (e.g. similarity search to autosuggest attributes based on colocated clusters)


In [ ]:
store = vector_store()